<a href="https://colab.research.google.com/github/rpast/t-sunset/blob/main/tsunset_finetuning_poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat Jan 21 21:43:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    24W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    print("no CUDA!")

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 8.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Import section ##
####################
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px 

import torch
from torch.utils.data import TensorDataset

from transformers import GPT2LMHeadModel, AdamW, GPT2Tokenizer

from sklearn.model_selection import train_test_split


In [ ]:
# Instatiate tokenizer
tokenizer = tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
#load dataset
dset = pd.read_csv('./drive/MyDrive/datasets/the-office_lines.csv')

dset_n = dset[['Character', 'Line']].copy()

# form inputs feature
dset_n['inputs'] = "CHARACTER: " + dset_n['Character'] + "; LINE: " + dset_n['Line'] + "<|endoftext|>"

dset_main = dset_n['inputs']

In [ ]:
# Split the data into a training and validation set with a ratio of 80:10
train_data, val_data = train_test_split(dset_main, test_size=0.1, random_state=42)

In [ ]:
# Load the pre-trained model
model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

# Define the optimizer and the learning rate
optimizer = AdamW(model.parameters(), lr=2e-5)

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
#inspect tokenizer varnames
tokenizer.batch_encode_plus.__code__.co_varnames

('self',
 'batch_text_or_text_pairs',
 'add_special_tokens',
 'padding',
 'truncation',
 'max_length',
 'stride',
 'is_split_into_words',
 'pad_to_multiple_of',
 'return_tensors',
 'return_token_type_ids',
 'return_attention_mask',
 'return_overflowing_tokens',
 'return_special_tokens_mask',
 'return_offsets_mapping',
 'return_length',
 'verbose',
 'kwargs',
 'padding_strategy',
 'truncation_strategy')

In [ ]:

# Tokenize the dataset
tokenizer.pad_token = tokenizer.eos_token
encoding = tokenizer.batch_encode_plus(
    train_data,
    pad_to_max_length=True,
    return_tensors="pt",
    return_token_type_ids=True,
    return_attention_mask=True,
    return_overflowing_tokens=True,
    return_special_tokens_mask=True,
    return_offsets_mapping=True,
    return_length=True
)

# Unpack the encoding dictionary to separate variables
input_ids = encoding["input_ids"]
token_type_ids = encoding["token_type_ids"]
attention_mask = encoding["attention_mask"]
overflowing_tokens = encoding["overflowing_tokens"]
special_tokens_mask = encoding["special_tokens_mask"]
offsets_mapping = encoding["offsets_mapping"]
length = encoding["length"]

# Create the TensorDataset
train_dataset = TensorDataset(
    input_ids,
    token_type_ids,
    attention_mask,
    overflowing_tokens,
    special_tokens_mask,
    offsets_mapping,
    length
    )

NotImplementedError: ignored

In [ ]:
# Tokenize the dataset
encoding = tokenizer.batch_encode_plus(
    val_data.tolist(),
    pad_to_max_length=True,
    return_tensors="pt",
    return_attention_mask=True,
    return_special_tokens_mask=True
)

# Convert the tokenized dataset to a TensorDataset
val_dataset = TensorDataset(
    encoding["input_ids"], 
    encoding["attention_mask"], 
    encoding["special_tokens_mask"]
)

In [ ]:
encoding.keys()

dict_keys(['input_ids', 'attention_mask', 'special_tokens_mask'])

In [ ]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy = "steps",    # evaluate every 1000 steps
    num_train_epochs=1,              # number of training epochs
    per_device_train_batch_size=2,  # batch size
    save_steps=1000                # save checkpoint every 1000 steps
)

# Initialize the trainer
trainer = Trainer(
    model=model,                         # the model to train
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset       # evaluation dataset
)

In [ ]:

# Start fine-tuning
trainer.train()

***** Running training *****
  Num examples = 52848
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 26424
  Number of trainable parameters = 124439808


TypeError: ignored

In [ ]:
tokenizer.batch_encode_plus.__code__.co_varnames

('self',
 'batch_text_or_text_pairs',
 'add_special_tokens',
 'padding',
 'truncation',
 'max_length',
 'stride',
 'is_split_into_words',
 'pad_to_multiple_of',
 'return_tensors',
 'return_token_type_ids',
 'return_attention_mask',
 'return_overflowing_tokens',
 'return_special_tokens_mask',
 'return_offsets_mapping',
 'return_length',
 'verbose',
 'kwargs',
 'padding_strategy',
 'truncation_strategy')

In [ ]:
import inspect

# Get the number of input arguments for the forward method
num_inputs = len(inspect.signature(GPT2LMHeadModel.forward).parameters)
print("Number of input tensors required by the model:", num_inputs)


Number of input tensors required by the model: 15


In [ ]:
training_args.per_device_train_batch_size

2

In [ ]:
print(train_dataset[0][0].shape)


torch.Size([359])


In [ ]:
print(model.config)

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.25.1",
  "use_cache": true,
  "vocab_size": 50257
}

